#### 📥 Librerías:

In [14]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
import matplotlib.pyplot as plt

### 📌 Parte 1: Preparación de Datos y Desarrollo del Modelo

✅ *Hemos descargado y guardado los datos en el archivo* `data/NVDA_10yr.csv`.  
El siguiente paso es normalizar la columna `Close` para preparar los datos para el modelo LSTM.


### 🎹 Paso 2: Normalización de datos
Para preparar los datos para la red LSTM, normalizaremos la columna `Close` usando `MinMaxScaler`, escalando los valores entre 0 y 1.

Guardaremos tanto los datos normalizados como el objeto `scaler` para reutilizarlo más adelante.

In [11]:
#  Cargar CSV con MultiIndex en columnas
df = pd.read_csv("data/NVDA_10yr.csv", header=[0, 1], index_col=0, parse_dates=True)

#  Eliminar MultiIndex: nos quedamos con el primer nivel del encabezado
df.columns = df.columns.get_level_values(0)

#  Normalizar la columna 'Close'
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close = scaler.fit_transform(df[["Close"]])

#  Guardar el scaler para reutilizarlo después
joblib.dump(scaler, "data/close_price_scaler.save")

#  Agregar columna normalizada al DataFrame
df["Close_Scaled"] = scaled_close

#  Mostrar un preview de los datos normalizados
df[["Close", "Close_Scaled"]].head(10)

Price,Close,Close_Scaled
Date,,
2014-01-02,0.373906,0.000079
2014-01-03,0.369427,0.000049
2014-01-06,0.374377,0.000083
2014-01-07,0.380507,0.000124
2014-01-08,0.385694,0.000159
2014-01-09,0.371313,0.000062
2014-01-10,0.370841,0.000059
2014-01-13,0.362118,0.000000
2014-01-14,0.373434,0.000076


 *La columna `Close` ha sido normalizada y guardamos el scaler en* `data/close_price_scaler.save`.  
Ahora toca hacer son **las secuencias de datos** para alimentar el modelo LSTM.


### 🩻 Paso 3: Crear secuencias para LSTM

Las redes requieren datos en forma de secuencias para capturar dependencias temporales. Usaremos un *look-back window* de **60 días**, lo que significa que el modelo utilizará los 60 precios anteriores para predecir el siguiente.

También dividiremos los datos en conjuntos de entrenamiento y prueba manteniendo el orden temporal (`shuffle=False`).

In [13]:
#  Definir función para crear secuencias
def create_sequences(data, look_back=60):
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i-look_back:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

#  Extraer la columna normalizada
scaled_data = df["Close_Scaled"].values.reshape(-1, 1)

#  Crear secuencias
look_back = 60
X, y = create_sequences(scaled_data, look_back=look_back)

#  Redimensionar X para LSTM: (samples, time_steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

#  Mostrar shapes
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

#  Dividir en entrenamiento y prueba (80% / 20%, sin shuffle)
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print("Shape X_train:", X_train.shape)
print("Shape X_test :", X_test.shape)


Shape de X: (2707, 60, 1)
Shape de y: (2707,)
Shape X_train: (2165, 60, 1)
Shape X_test : (542, 60, 1)


### 🧠Paso 4: Construcción del modelo LSTM

Crearemos dos arquitecturas de red neuronal:

1. 🔹 *Modelo sencillo:* una capa LSTM con Dropout para prevenir overfitting y una capa Dense de salida.
2. 🔹 *Modelo robusto:* dos capas LSTM apiladas con Dropout y una capa Dense.

Usaremos `Keras` con TensorFlow como backend.

### 🔹 Modelo 1: Sencillo (1 LSTM)

In [15]:
#  Construir el modelo sencillo
model_simple = Sequential([
    LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    Dense(1)  # Capa de salida
])

#  Compilar el modelo
model_simple.compile(optimizer="adam", loss="mean_squared_error")

#  Resumen del modelo
model_simple.summary()


/Users/carlosmoreno/.pyenv/versions/3.10.12/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

### 🔹 Modelo 2: Robusto (2 LSTM)

In [16]:
# 📦 Construir el modelo robusto
model_robust = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)  # Capa de salida
])

# 🛠 Compilar el modelo
model_robust.compile(optimizer="adam", loss="mean_squared_error")

# 📃 Resumen del modelo
model_robust.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,651 (119.73 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

En el siguiente paso entrenaremos ambos modelos y compararemos sus predicciones con los valores reales.